## 상세페이지 크롤링

In [21]:
import requests
from bs4 import BeautifulSoup
from datetime import date, timedelta
import pandas as pd
import time
import re

In [22]:
#category, sub_category 맨 앞 0을 인식 못해 str으로 바꿔줌.
data = pd.read_csv('mini_onepiece(020006)_210324.csv',index_col = 0,converters={'category':str,'sub_category':str})

data.head()

,item_id,img_url,sub_category,category,page_num,date
0,1854456,https://image.msscdn.net/images/goods_img/2021...,020006,020,0,2021-03-24
1,1854413,https://image.msscdn.net/images/goods_img/2021...,020006,020,0,2021-03-24
2,1854457,https://image.msscdn.net/images/goods_img/2021...,020006,020,0,2021-03-24
3,1854414,https://image.msscdn.net/images/goods_img/2021...,020006,020,0,2021-03-24
4,1854458,https://image.msscdn.net/images/goods_img/2021...,020006,020,0,2021-03-24


In [5]:
data['item_id'].unique()

array([1854456, 1854413, 1854457, ..., 1559214, 1792848, 1801848])

In [6]:
url = 'https://store.musinsa.com/app/goods/' + '1854456'

req = requests.get(url)
raw = req.text

html = BeautifulSoup(raw, 'html.parser')

In [81]:
title = html.select('span.product_title')[0].get_text() #'\n에어리핏 슬릿 원피스_퍼플헤더_(AKPOP-15LVD)\n'
season = html.select('#product_order_info > div.explan_product.product_info_section > ul > li:nth-child(2) > p.product_article_contents > strong')[0].get_text()
gender = html.select('span.txt_gender')[0].get_text() #'\n여\n'
brand_product_num=html.select('p.product_article_contents')[0].get_text() #brand/품번
tag = html.select('a.listItem') #상품 특성 키워드로#트레이닝복
price =html.select('span.product_article_price')[0].get_text()#\n35,000원\n 
#mem_price = html.select('div.member_price')
likes = html.select('a.goods_like_1854456')[0].get_text() #'좋아요0'

In [ ]:
import re

string = "Hey! What's up?"
string = re.sub("\!|\'|\?","",string)
print(string) 

In [84]:
re.sub('\ |\\n',"",season)

'2021S/S'

In [107]:
title = re.sub('\\n',"",title)
season = re.sub('\ |\\n',"",season)
gender = re.sub('\\n',"",gender)
brand = brand_product_num.split()[0]
product_num = brand_product_num.split()[2]
tag_l = []
for i in tag:
    tag_l.append(re.sub('\#','',i.get_text()))
tag = tag_l
price = re.sub('\ |\\n|\,|\원','',price)
likes = likes[3:]

In [37]:
title_c=[]
season_c=[]
gender_c=[]
brand_c=[]
product_num_c = []
tag_c = []
price_c = []
likes_c =[]

In [36]:
goods_like_1854456

1

In [39]:
start = time.time()  # 시작 시간 저장

for i in range(len(data)):
    item_id = data['item_id'][i]
    url = 'https://store.musinsa.com/app/goods/' + str(item_id)
    
    req = requests.get(url)
    raw = req.text
    html = BeautifulSoup(raw, 'html.parser')
    
    title = html.select('span.product_title')[0].get_text() #'\n에어리핏 슬릿 원피스_퍼플헤더_(AKPOP-15LVD)\n'
    season = html.select('#product_order_info > div.explan_product.product_info_section > ul > li:nth-child(2) > p.product_article_contents > strong')[0].get_text()
    gender = html.select('span.txt_gender')[0].get_text() #'\n여\n'
    brand_product_num=html.select('p.product_article_contents')[0].get_text() #brand/품번
    tag = html.select('a.listItem') #상품 특성 키워드로#트레이닝복 #운동복 
    price =html.select('span.product_article_price')[0].get_text()#\n35,000원\n 
    #mem_price = html.select('div.member_price')
    likes = html.select('a.goods_like_'+str(item_id))[0].get_text() #'좋아요0'

    title_c.append(re.sub('\\n',"",title))
    season_c.append(re.sub('\ |\\n',"",season))
    gender_c.append(re.sub('\\n',"",gender))
    brand_c.append(brand_product_num.split()[0])
    product_num_c.append(brand_product_num.split()[2])
    tag_l = []
    for i in tag:
        tag_l.append(re.sub('\#','',i.get_text()))
    tag_c.append(tag_l)
    price_c.append(re.sub('\ |\\n|\,|\원','',price))
    likes_c.append(likes[3:])
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

IndexError: list index out of range

In [41]:
#like 이전꺼 +1 더 되어있을 수 있음.
print(len(title_c),
len(season_c),
len(gender_c),
len(brand_c),
len(product_num_c),
len(tag_c),
len(price_c),
len(likes_c))

301 301 301 301 301 301 301 301


In [45]:
df = pd.DataFrame()
df['item_id'] = data['item_id'][:301]
df['title'] = title_c
df['season'] = season_c
df['gender'] = gender_c
df['brand'] = brand_c
df['product'] = product_num_c
df['tag']= tag_c
df['price'] =price_c
df['likes'] = likes_c
df['date'] = date.today().isoformat() #'yyyy-mm-dd'

In [49]:
df.head(10)

,item_id,title,season,gender,brand,product,tag,price,likes,date
0,1854456,에어리핏 슬릿 원피스_퍼플헤더_(AKPOP-15LVD),2021S/S,여,ANDAR,AKPOP-15LVD,"[트레이닝복, 운동복, 요가복, 애슬레저룩, 애슬레저, 필라테스복, 운동하의]",35000,0,2021-03-24
1,1854413,에어리핏 슬릿 원피스_오프화이트_(AKPOP-15WHT),2021S/S,여,ANDAR,AKPOP-15WHT,"[트레이닝복, 운동복, 요가복, 애슬레저룩, 애슬레저, 필라테스복, 운동하의]",35000,0,2021-03-24
2,1854457,에어리핏 슬릿 원피스_블룸핑크_(AKPOP-15LPK),2021S/S,여,ANDAR,AKPOP-15LPK,"[트레이닝복, 운동복, 요가복, 애슬레저룩, 애슬레저, 필라테스복, 운동하의]",35000,0,2021-03-24
3,1854414,에어리핏 슬릿 원피스_랍스타핑크_(AKPOP-15PNK),2021S/S,여,ANDAR,AKPOP-15PNK,"[트레이닝복, 운동복, 요가복, 애슬레저룩, 애슬레저, 필라테스복, 운동하의]",35000,0,2021-03-24
4,1854458,에어리핏 슬릿 원피스_오이스터차콜_(AKPOP-15CHA),2021S/S,여,ANDAR,AKPOP-15CHA,"[트레이닝복, 운동복, 요가복, 애슬레저룩, 애슬레저, 필라테스복, 운동하의]",35000,0,2021-03-24
5,1854415,에어리핏 슬릿 원피스_딥위스_(AKPOP-15NAV),2021S/S,여,ANDAR,AKPOP-15NAV,"[트레이닝복, 운동복, 요가복, 애슬레저룩, 애슬레저, 필라테스복, 운동하의]",35000,0,2021-03-24
6,1854459,에어리핏 슬릿 원피스_메티오블랙_(AKPOP-15BLK),2021S/S,여,ANDAR,AKPOP-15BLK,"[트레이닝복, 운동복, 요가복, 애슬레저룩, 애슬레저, 필라테스복, 운동하의]",35000,0,2021-03-24
7,1854439,에어리핏 슬릿 원피스_아니스옐로우_(AKPOP-15LYW),2021S/S,여,ANDAR,AKPOP-15LYW,"[트레이닝복, 운동복, 요가복, 애슬레저룩, 애슬레저, 필라테스복, 운동하의]",35000,0,2021-03-24
8,1854239,엠마 트위드 체크 드레스_블랙Emma Tweed Check Dress_Black,2021S/S,여,JUDE,/,"[여성원피스, 추천원피스, 캐주얼원피스, 여자원피스, 봄원피스, 미니원피스]",170000,0,2021-03-24
9,1854102,엠마 트위드 체크 드레스_아이보리Emma Tweed Check Dress_Ivory,2021S/S,여,JUDE,/,"[캐주얼원피스, 드레스, 봄원피스, 미니원피스, 원피스, 트위드, 트위드원피스]",170000,0,2021-03-24


In [50]:
df.to_csv('mini_onepiece_detail(020006)_210324.csv')